In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')

In [ ]:
# %cd "/gdrive/My Drive/Colab Notebooks/"
# %tensorflow_version 1.x
# import tensorflow
# print("tensorflow version :", tensorflow.__version__)

In [ ]:
from keras.models import Model
from keras.utils import Sequence
from keras.layers import Reshape, Input, Activation, Conv2D, GlobalMaxPool2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Flatten, Dense, Lambda, ZeroPadding2D, Dropout, DepthwiseConv2D
from keras.layers.advanced_activations import LeakyReLU, ReLU
from keras.layers.merge import concatenate
from keras.optimizers import SGD, Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

from imgaug import augmenters as iaa
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pickle
import copy
import cv2
import os

from Yolo_V2_utils import *
from Yolo_V2_extractors import *
from Yolo_V2_preprocessing import *
from Yolo_V2_detection_head import *

%matplotlib inline

In [ ]:
OBJECT = 'Plate' # Plate
DATASETS = ['GENERATED_DATASET'] # GENERATED, MOROCCAN, MIXED
BACKEND = 'SqueezeNet' # DarkNet, MobileNet, SqueezeNet

ROOT_DIR = '/plate-detector/'
DATA_DIR = ROOT_DIR + 'data/'
MODELS_DIR = ROOT_DIR + 'models/'
BACKEND_DIR = ROOT_DIR + 'backends/'

# l'entrainement des extracteurs prend beaucoup de temps, alors j'ai choisi d'utilisée l'apprentissage transféré,
# c-à-d commencer l'entrainement par les poids des structures déja entrainées sur la classification des images 
# dans le concours ImageNet Object Classification.
SQUEEZENET_BACKEND_PATH = BACKEND_DIR + "squeezenet_backend.h5"
MOBILENET_BACKEND_PATH  = BACKEND_DIR + "mobilenet_backend.h5"

TRAIN_VALID_PROP = 0.8
INPUT_SIZE = 320          # maximum width or height
LABELS = [OBJECT]         # Liste des objets que doit YOLO considérer lors de la détection et entrainement
MAX_BOX_PER_IMAGE = 1     # GENERATED 1, MOROCCAN 26
OBJ_THRESH = 0.5          # Valeur minimale de sûreté/fidélité/confidence pour considérer la détéction positive
NMS_THRESH = 0.5          # Valeur minimale de sûreté/fidélité/confidence pour considérer la détéction positive
ANCHORS = [6.45,2.55, 6.54,4.02, 8.37,5.55, 10.17,3.91, 10.66,7.32, 13.16,5.34, 13.47,9.36, 14.92,7.32, 15.33,11.66]

In [ ]:
all_imgs, seen_labels = parse_annotation(DATA_DIR, DATASETS)

train_valid_split = int(TRAIN_VALID_PROP * len(all_imgs))
np.random.shuffle(all_imgs)
valid_imgs = all_imgs[train_valid_split:]
train_imgs = all_imgs[:train_valid_split]

print(seen_labels)

In [ ]:
# def dic_save(dic, dic_name):
#     with open(dic_name + '.pickle', 'wb') as f:
#         pickle.dump(dic, f, pickle.HIGHEST_PROTOCOL)
#     print("saved")

# def dic_load(dic_name):
#     with open(dic_name + '.pickle', 'rb') as f:
#         return pickle.load(f)

# all_imgs = dic_load("all_imgs")
# print(len(all_imgs))
# train_valid_split = int(TRAIN_VALID_PROP * len(all_imgs))
# np.random.shuffle(all_imgs)
# valid_imgs = all_imgs[train_valid_split:]
# train_imgs = all_imgs[:train_valid_split]

In [ ]:
# Visualisation des données
fig=plt.figure(figsize=(20, 20))
columns = 4
rows = 1
for i in range(1, columns*rows +1):
    numb = np.random.randint(len(train_imgs))
    rand_img = cv2.imread(train_imgs[numb]['file_name'])
    rand_img = cv2.cvtColor(rand_img, cv2.COLOR_BGR2RGB)
    for j in range(len(train_imgs[numb]['object'])) :
        min_labels = (train_imgs[numb]['object'][j]['xmin'], train_imgs[numb]['object'][j]['ymin'])
        max_labels = (train_imgs[numb]['object'][j]['xmax'], train_imgs[numb]['object'][j]['ymax'])
        cv2.rectangle(rand_img, min_labels, max_labels, (0,255,0), 2)
    fig.add_subplot(rows, columns, i)
    plt.imshow(rand_img)  
plt.show()

In [ ]:
model = YOLO(BACKEND, INPUT_SIZE, LABELS, MAX_BOX_PER_IMAGE, ANCHORS)

In [ ]:
model.train(   train_imgs         = train_imgs, # Les images sur lesquelles le model sera entrainé
               valid_imgs         = valid_imgs, # Les images sur lesquelles la performance du model sera validé
               train_times        = 8, # Le nombre de fois pour répéter l'entrainement
               valid_times        = 1, # Le nombre de fois pourrépéter la validation
               nb_epochs          = 500, # Le nombre d'époques
               learning_rate      = 1e-4, # Le taux d'apprentissage initial : \ethat_{0}
               batch_size         = 16, # La taille du batch : Puisqu'on utilise un entrainement par batch
               warmup_epochs      = 3, # Des époques pour l'échauffement permet la familiarisation avec les données
               object_scale       = 5, # Pénalité sur les fausse négative : un objet n'est pas détecté
               no_object_scale    = 1, # Pénalité sur les fausse positive : une détection d'objets inexistant
               coord_scale        = 1, # Pénalité sur les coordonnées du centre de l'objet
               class_scale        = 1, # Pénalité sur les dimensions de l'objet détecté
               saved_weights_dir  = MODELS_DIR,
               debug              = True)